In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
%matplotlib inline

In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            -46.65810255733541,
            -23.535732559501554
          ],
          [
            -46.603706283614684,
            -23.535732559501554
          ],
          [
            -46.603706283614684,
            -23.575921045095285
          ],
          [
            -46.65810255733541,
            -23.575921045095285
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

In [ ]:
ffa_db.bandNames().getInfo()


In [ ]:
url = ffa_db.select(['B2','B3','B4']).getThumbURL({'min': 0, 'max': 3000})
disp.Image(url=url, width=800)